In [ ]:
import os
import numpy as np
import pandas as pd
import xarray as xr
import math

import esdglider.gcp as gcp
import esdglider.glider as glider
import esdglider.utils as utils

deployment_info = {
    "deployment": 'calanus-20241019', 
    "project": "ECOSWIM", 
    "mode": 'delayed', 
    "min_dt": '2024-10-19 17:37:00', 
}
deployment = deployment_info["deployment"]
project = deployment_info["project"]
mode = deployment_info["mode"]

year = utils.year_path(project, deployment)
bucket_name = 'amlr-gliders-deployments-dev'
deployments_path = f"/home/sam_woodman_noaa_gov/{bucket_name}"
config_path = f"/home/sam_woodman_noaa_gov/glider-lab/deployment-config"

gcp.gcs_mount_bucket(
    "amlr-gliders-deployments-dev", deployments_path, 
    ro=False)


paths = glider.get_path_deployment(
    deployment_info, deployments_path, config_path)

dir_ts = paths["tsdir"]
path_sci = os.path.join(dir_ts, f"{deployment}-{mode}-sci.nc")
path_eng = os.path.join(dir_ts, f"{deployment}-{mode}-eng.nc")

dir_ngdac = paths["profdir"]
if os.path.isdir(dir_ngdac):
    files_ngdac = os.listdir(dir_ngdac)
# files_ngdac
# os.path.join(paths["profdir"], f"{deployment} -{mode}.nc")
# path_sci = os.path.join(dir_ts, 'calanus-20241019-sci.nc')
# path_eng = os.path.join(dir_ts, 'calanus-20241019-eng.nc')

path_g5 = os.path.join(paths["griddir"], f"{deployment}_grid-{mode}-5m.nc")
ds_eng = xr.load_dataset(path_eng)
ds_sci = xr.load_dataset(path_sci)


In [ ]:
ds_raw = xr.load_dataset(os.path.join(paths["rawdir"], f"{deployment}-{mode}-raw.nc"))
x0 = ds_raw.sel(time=slice("2024-11-01 18:00", "2024-11-01 22:00"))

# from pyglider import utils as pgutils

# x2 = pgutils.get_glider_depth(x)
# x2
x0

In [ ]:
print(np.max(ds_sci.depth.values))
print(np.argmax(ds_sci.depth.values))
ds_sci.time[516252]

x = ds_sci.sel(time=slice("2024-11-01 18:00", "2024-11-01 22:00"))

# from pyglider import utils as pgutils

# x2 = pgutils.get_glider_depth(x)
# x2
x

In [ ]:
y = ds_eng.sel(time=slice("2024-11-01 18:00", "2024-11-01 22:00"))
y

In [ ]:
from esdglider import plots
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
# from matplotlib_scalebar.scalebar import ScaleBar

bar_file = "/home/sam_woodman_noaa_gov/ETOPO_2022_v1_15s_N45W135_erddap.nc"
bar = xr.load_dataset(bar_file).rename({"latitude": "lat", "longitude": "lon"})
bar = bar.where(bar.z <= 0, drop=True)

path_g5 = os.path.join(paths["griddir"], f"{deployment}_grid-{mode}-5m.nc")
ds = xr.load_dataset(path_g5)
plots.sci_surface_map(
    ds, 
    "temperature", 
    crs=ccrs.Mercator(), 
    show=False, 
    bar=None, 
)

# # plt.show()

deployment = ds.deployment_name
# start = ds.deployment_start[0:10]
# end = ds.deployment_end[0:10]

# map_lon_border = 0.1
# map_lat_border = 0.2
# glider_lon_min = ds.longitude.min()
# glider_lon_max = ds.longitude.max()
# glider_lat_min = ds.latitude.min()
# glider_lat_max = ds.latitude.max()
    
# # Cartopy
# fig, ax = plt.subplots(
#     figsize=(8.5, 11), subplot_kw={"projection": ccrs.Mercator()}
# )
# ax.set_xlabel("\n\n\nLongitude [Deg]", size=14)
# ax.set_ylabel("Latitude [Deg]\n\n\n", size=14)

# # Set extent of the map based on the glider data
# ax.set_extent(
#     [
#         glider_lon_min - map_lon_border,
#         glider_lon_max + 3 * map_lon_border,
#         glider_lat_min - map_lat_border,
#         glider_lat_max + map_lat_border,
#     ],
#     crs=ccrs.PlateCarree(),
# )

# # Add coastlines, countries, and continents
# # ax.coastlines(resolution="10m", color="black")
# # ax.add_feature(cfeature.BORDERS, linestyle="-", edgecolor="black")
# ax.add_feature(cfeature.LAND, edgecolor="black", facecolor="#e0b479")
# ax.add_feature(cfeature.OCEAN, edgecolor="none", facecolor="#7bcbe3")

# gl = ax.gridlines(draw_labels=["bottom", "left"])
# gl.xlabel_style = {'rotation': 15}

# # for label in ax.get_xticklabels():
# #     label.set(rotation=-30)

# # plt.show()

# # ax.add_artist(ScaleBar(1, location="lower right"))


# plots.sci_timeseries_loop(ds_sci, None, show=True)
plots.eng_timeseries_loop(ds_eng, None, show=True)